In [1]:
!pip install -q transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from transformers import pipeline

In [3]:
df = pd.read_csv('/content/text_classification_dataset_extended.csv')

In [4]:
label_encoder = LabelEncoder()

df['label'] = label_encoder.fit_transform(df['class'])

class_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

In [5]:
df.head()

,text,class,label
0,Local team wins the championship!,sports,4
1,Stock market reaches all-time highs.,finance,0
2,Politician announces new policy.,political,3
3,City council approves new park.,local,2
4,International conference held in Paris.,international,1


In [7]:
df.shape

(40, 3)

In [8]:
dataset = Dataset.from_pandas(df[['text','label']])
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 40
})

In [9]:
dataset[0]

{'text': 'Local team wins the championship!', 'label': 4}

In [10]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [11]:
tokenized_text = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
label2id = {'finance': 0, 'international': 1, 'local': 2, 'political': 3, 'sports': 4}
id2label = {v: k for k, v in label2id.items()}

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=5, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
training_args = TrainingArguments(
    output_dir="my_model",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    max_steps = 200,
    save_steps = 20,
    logging_steps = 5
)


In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_text,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
5,1.599000
10,1.580000
15,1.536700
20,1.434700
25,1.388500
30,1.261600
35,1.132400
40,1.021100
45,0.905200
50,0.770800


TrainOutput(global_step=200, training_loss=0.4558295960724354, metrics={'train_runtime': 281.0395, 'train_samples_per_second': 2.847, 'train_steps_per_second': 0.712, 'total_flos': 2171339618160.0, 'train_loss': 0.4558295960724354, 'epoch': 20.0})

In [18]:
text = "Pakistan have joined India at the top of the ICC World Test Championship standings with a comprehensive four-wicket victory over Sri Lanka in the first Test"

classifier = pipeline(task="text-classification", model="/content/my_model/checkpoint-80")
classifier(text)

[{'label': 'sports', 'score': 0.3122710883617401}]